In [123]:
import pandas as pd
from config import data_path, fixed_service_schedule_fn, truck_schedule_fn, possible_paths_fn
from model_input import loading_time_window, storage_cost, barge_handling_cost, train_handling_cost, truck_handling_cost

In [124]:
# Load the CSV file into a DataFrame
data = pd.read_csv(f"{data_path}\\{fixed_service_schedule_fn}")
truck_df = pd.read_csv(f"{data_path}\\{truck_schedule_fn}")

In [125]:
# Constants
LOADING_TIME = loading_time_window/60  # Loading time in hours
TRANSHIPMENT_TIME = 2*LOADING_TIME # Loading + Unloading time
TRANSSHIPMENT_COST_BARGE = barge_handling_cost # Cost per transshipment
TRANSSHIPMENT_COST_TRAIN = train_handling_cost # Cost per transshipment
TRANSSHIPMENT_COST_TRUCK = truck_handling_cost # Cost per transshipment
STORAGE_COST_PER_HOUR = storage_cost  # Storage cost per hour of waiting


In [126]:
def find_paths_recursive(origin, destination, services, current_path=[], current_services=[]):
    """
    Recursively find all paths from origin to destination from a list of service dictionaries.
    """
    if origin == destination:
        return [{'path': current_path, 'services': current_services}]
    
    paths = []
    possible_services = [service for service in services if service['Origin'] == origin]

    for service in possible_services:
        # Check if this is the first service or if the service is temporally feasible
        if not current_services or service['Departure'] >= current_services[-1]['Arrival'] + LOADING_TIME + TRANSHIPMENT_TIME:
            next_paths = find_paths_recursive(service['Destination'], destination, services,
                                              current_path + [(origin, service['Destination'])],
                                              current_services + [service])
            paths.extend(next_paths)
    
    return paths

In [127]:
def calculate_costs_and_emissions(paths):
    """
    Calculate total cost and emissions for each path.
    """
    for path in paths:
        total_transport_cost = sum(service['Travel Cost'] for service in path['services'])
        # total_emission = sum(service['carbonemissions'] for service in path['services'])
        # transshipment_frequency = (len(path['services']) - 1 if len(path['services']) > 1 else 0)
        # transshipment_cost = TRANSSHIPMENT_COST * transshipment_frequency
        transshipment_cost = 0
        n = 0
        if len(path['services']) > 1:
            for service in path['services']:
                mode = service['Mode']
                if mode == 'Barge':
                    hc = TRANSSHIPMENT_COST_BARGE
                elif mode == 'Train':
                    hc = TRANSSHIPMENT_COST_TRAIN
                elif mode == 'Truck':
                    hc = TRANSSHIPMENT_COST_TRUCK

                if n > 0:
                    transshipment_cost += 2 * hc
                else:
                    transshipment_cost += hc
                n += 1
            transshipment_cost -= hc # remove the unloading in the destination terminal
        
        storage_cost = 0
        for i in range(1, len(path['services'])):
            waiting_time = path['services'][i]['Departure'] - path['services'][i-1]['Arrival'] - TRANSHIPMENT_TIME
            storage_cost += max(0, waiting_time) * STORAGE_COST_PER_HOUR

        transshipment_times = len(path['services']) - 1

        total_cost = total_transport_cost + transshipment_cost + storage_cost
        path.update({
            'total_transport_cost': total_transport_cost,
            # 'total_emission': total_emission,
            'transshipment_cost': transshipment_cost,
            'storage_cost': storage_cost,
            'total_cost': total_cost,
            'transshipment_times': transshipment_times
        })

    return paths

In [128]:
# Convert services to a list of dicts for easier manipulation in recursive calls
services_list = data.to_dict('records')

# Example usage
origin = "Delta"
destination = 'Venlo'
paths = find_paths_recursive(origin, destination, services_list)
calculated_paths = calculate_costs_and_emissions(paths)

# Print the calculated details for all paths
for path in calculated_paths:
    print(path)

{'path': [('Delta', 'Euromax'), ('Euromax', 'Venlo')], 'services': [{'Service_ID': 'Barge01', 'Mode': 'Barge', 'Origin': 'Delta', 'Destination': 'Euromax', 'Departure': 53, 'Arrival': 54.0, 'Travel Time': 2.0, 'Capacity': 160, 'Speed': 15, 'Travel Cost': 0.9317}, {'Service_ID': 'Barge36', 'Mode': 'Barge', 'Origin': 'Euromax', 'Destination': 'Venlo', 'Departure': 75, 'Arrival': 88.5, 'Travel Time': 14.5, 'Capacity': 160, 'Speed': 15, 'Travel Cost': 12.57795}], 'total_transport_cost': 13.509649999999999, 'transshipment_cost': 6.0, 'storage_cost': 18.0, 'total_cost': 37.50965, 'transshipment_times': 1}
{'path': [('Delta', 'Euromax'), ('Euromax', 'Venlo')], 'services': [{'Service_ID': 'Barge01', 'Mode': 'Barge', 'Origin': 'Delta', 'Destination': 'Euromax', 'Departure': 53, 'Arrival': 54.0, 'Travel Time': 2.0, 'Capacity': 160, 'Speed': 15, 'Travel Cost': 0.9317}, {'Service_ID': 'Train17', 'Mode': 'Train', 'Origin': 'Euromax', 'Destination': 'Venlo', 'Departure': 78, 'Arrival': 82.5, 'Travel

In [129]:
def remove_origin_revisiting_paths(paths):
    """
    Removes paths that revisit the origin after the first node in the path sequence.
    """
    filtered_paths = []
    for path in paths:
        # Get a list of all stops after the first one
        stops_after_first = [step[1] for step in path['path'][1:]]
        
        # Find the origin of the current path (first node)
        path_origin = path['path'][0][0]
        
        # If the origin is not revisited, add the path to the filtered list
        if path_origin not in stops_after_first:
            filtered_paths.append(path)
    
    return filtered_paths

# Assuming 'calculated_paths' contains the list of paths with calculated costs and emissions
# Apply the filter to the calculated paths
filtered_paths = remove_origin_revisiting_paths(calculated_paths)


In [130]:
# Get all unique origins and destinations
origins = data['Origin'].unique()
destinations = data['Destination'].unique()

# Container for all filtered paths
all_filtered_paths = []

# Iterate over all unique pairs of origin and destination
for origin in origins:
    for destination in destinations:
        if origin != destination:
            # Calculate all paths for the current origin-destination pair
            paths = find_paths_recursive(origin, destination, data.to_dict('records'))
            calculated_paths = calculate_costs_and_emissions(paths)
            
            # Filter out the paths revisiting the origin
            filtered_paths = remove_origin_revisiting_paths(calculated_paths)
            
            # Add the filtered paths to the container
            all_filtered_paths.extend(filtered_paths)

# Prepare the data for CSV output
data_for_csv = [{
    'origin': path['path'][0][0],
    'destination': path['path'][-1][-1],
    'path': ' -> '.join(f"{step[0]} to {step[1]}" for step in path['path']),
    'service_ids': ', '.join(str(service['Service_ID']) for service in path['services']),
    'first_service_mode': path['services'][0]['Mode'],
    'last_service_mode': path['services'][-1]['Mode'],
    'first_service':path['services'][0]['Service_ID'],
    'first_service_departure': path['services'][0]['Departure'],  # First service departure time
    'service_capacities': ', '.join(str(service['Capacity']) for service in path['services']),
    'service_capacity': min((service['Capacity']) for service in path['services']),
    'total_transport_cost': path['total_transport_cost'],
    # 'total_emission': path['total_emission'],

    'transshipment_cost': path['transshipment_cost'],
    'transshipment_time': path['transshipment_times'],
    'storage_time': path['storage_cost']/STORAGE_COST_PER_HOUR,
    'storage_cost': path['storage_cost'],
    'total_cost': path['total_cost']
} for path in all_filtered_paths]

# Create a DataFrame from the filtered data
df_all_filtered_paths = pd.DataFrame(data_for_csv)
# Write the DataFrame to a single CSV file
# df_all_filtered_paths.to_csv('total1.csv', index=False)


Post-Processing to Add Trucks

In [131]:
# Function to get truck travel time
def get_truck_travel_time(origin, destination):
    row = truck_df[(truck_df['Origin'] == origin) & (truck_df['Destination'] == destination)]
    if not row.empty:
        return row['Travel Time'].values[0]
    return None

# Function to get truck travel cost
def get_truck_travel_cost(origin, destination):
    row = truck_df[(truck_df['Origin'] == origin) & (truck_df['Destination'] == destination)]
    if not row.empty:
        return row['Travel Cost'].values[0]
    return None

# Function to get service departure time
def get_service_departure(service_id):
    row = data[data['Service_ID'] == service_id]
    if not row.empty:
        return row['Departure'].values[0]
    return None

# Function to get service arrival time
def get_service_arrival(service_id):
    row = data[data['Service_ID'] == service_id]
    if not row.empty:
        return row['Arrival'].values[0]
    return None

# Function to get service origin
def get_service_origin(service_id):
    row = data[data['Service_ID'] == service_id]
    if not row.empty:
        return row['Origin'].values[0]
    return None

# Function to get service destination
def get_service_destination(service_id):
    row = data[data['Service_ID'] == service_id]
    if not row.empty:
        return row['Destination'].values[0]
    return None

# Function to get service arrival time
def get_service_cost(service_id):
    row = data[data['Service_ID'] == service_id]
    if not row.empty:
        return row['Travel Cost'].values[0]
    return None
def get_service_capacity(service_id):
    row = data[data['Service_ID'] == service_id]
    if not row.empty:
        return row['Capacity'].values[0]
    return None
def get_truck_ID(origin, destination):
    row = truck_df[(truck_df['Origin'] == origin) & (truck_df['Destination'] == destination)]
    if not row.empty:
        return row['Service_ID'].values[0]
    return None  # Optional: return None or handle the case where no match is found
def extract_intermediary_points(path):
    parts = path.split(' -> ')
    intermediaries = [part.split(' to ')[1] for part in parts[:-1]]
    return ','.join(intermediaries)

# # Function to correct transshipment costs
def correct_transshipment_costs(prev_trans_cost, prev_mode, barge_hc, train_hc, truck_hc, last_mode):
    # Define the cost for each mode
    costs = {
        'Barge': barge_hc,
        'Train': train_hc
    }
    
    if last_mode:
        new_trans_cost = prev_trans_cost - costs.get(prev_mode, 0) + truck_hc
    else:
        new_trans_cost = prev_trans_cost - 2*costs.get(prev_mode, 0) + 2*truck_hc
    return new_trans_cost


# Function to add loading and unloading costs
def add_loading_unloading_costs(df,barge_hc, train_hc, truck_hc):
    # Define the cost for each mode
    costs = {
        'Barge': barge_hc,
        'Train': train_hc,
        'Truck': truck_hc
    }

    # Add new columns based on the first and last service mode
    df['Loading_cost_at_origin'] = df['first_service_mode'].apply(lambda mode: costs.get(mode, 0))
    df['Unloading_cost_at_destination'] = df['last_service_mode'].apply(lambda mode: costs.get(mode, 0))



    return df

In [132]:
df

,origin,destination,path,service_ids,first_service_mode,last_service_mode,first_service,first_service_departure,service_capacities,service_capacity,total_transport_cost,transshipment_cost,transshipment_time,storage_time,storage_cost,total_cost
0,Delta,Euromax,Delta to Euromax,Barge01,Barge,Barge,Barge01,53,160,160,0.93170,0.0,0,0.0,0.0,0.93170
1,Delta,HOME,Delta to HOME,Barge02,Barge,Barge,Barge02,53,160,160,2.32925,0.0,0,0.0,0.0,2.32925
2,Delta,Moerdijk,Delta to Euromax -> Euromax to Moerdijk,"Barge01, Barge34",Barge,Barge,Barge01,53,"160, 160",160,6.05605,6.0,1,42.0,42.0,54.05605
3,Delta,Moerdijk,Delta to HOME -> HOME to Moerdijk,"Barge02, Barge42",Barge,Barge,Barge02,53,"160, 160",160,5.12435,6.0,1,42.5,42.5,53.62435
4,Delta,Moerdijk,Delta to Moerdijk,Barge03,Barge,Barge,Barge03,3,160,160,4.65850,0.0,0,0.0,0.0,4.65850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,Duisburg,Dortmund,Duisburg to Dortmund,Train32,Train,Train,Train32,121,90,90,15.71775,0.0,0,0.0,0.0,15.71775
371,Duisburg,Nuremberg,Duisburg to Nuremberg,Train33,Train,Train,Train33,122,90,90,104.78500,0.0,0,0.0,0.0,104.78500
372,Venlo,Neuss,Venlo to Neuss,Train29,Train,Train,Train29,112,90,90,15.71775,0.0,0,0.0,0.0,15.71775
373,Venlo,Dortmund,Venlo to Dortmund,Train30,Train,Train,Train30,113,90,90,26.19625,0.0,0,0.0,0.0,26.19625


In [133]:
# Process the dataset
new_rows = []
df = df_all_filtered_paths
truck_deaprture_window = 2.5
truck_loading_time  = 1.5

for index, row in df.iterrows():
    services = row['service_ids'].split(', ')
    if row['transshipment_time'] == 1:
        # Replace the second service with "Truck"
        new_row = row.copy()
        first_service_id = services[0]
        second_service_id = services[1]
        second_mode = second_service_id[:-2]
        updated_transshipment_cost = correct_transshipment_costs(row['transshipment_cost'], second_mode, TRANSSHIPMENT_COST_BARGE, TRANSSHIPMENT_COST_TRAIN, TRANSSHIPMENT_COST_TRUCK, True)
        first_service_arrival = get_service_arrival(first_service_id)
        truck_travel_time = get_truck_travel_time(row['origin'], row['destination'])
    
        if first_service_arrival is not None and truck_travel_time is not None:
            truck_ID = get_truck_ID(get_service_destination(first_service_id),row['destination'])
            new_row['service_ids'] = first_service_id + f', {truck_ID}'
            new_row['first_service_departure'] = row['first_service_departure']
            truck_departure = first_service_arrival + truck_deaprture_window 
            truck_arrival = truck_departure + truck_travel_time
            new_row['last_service_arrival'] = truck_arrival
            new_row['storage_time'] = truck_deaprture_window-truck_loading_time
            new_row['storage_cost'] = (truck_deaprture_window-truck_loading_time)*STORAGE_COST_PER_HOUR
            new_row['transshipment_cost'] = updated_transshipment_cost
            new_row['total_transport_cost'] = get_service_cost(first_service_id) + get_truck_travel_cost(get_service_destination(first_service_id),row['destination'])
            new_row['total_cost'] = new_row['total_transport_cost']+new_row['storage_cost']+row['transshipment_cost']
            new_row['service_capacity'] = get_service_capacity(first_service_id)
            new_row['last_service_mode'] = 'Truck'
            new_rows.append(new_row)
    
    elif row['transshipment_time'] == 2:
        # Replace the second service with "Truck"
        new_row = row.copy()
        first_service_id = services[0]
        second_service_id = services[1]
        third_service_id = services[2]
        second_mode = second_service_id[:-2]
        updated_transshipment_cost = correct_transshipment_costs(row['transshipment_cost'], second_mode, TRANSSHIPMENT_COST_BARGE, TRANSSHIPMENT_COST_TRAIN, TRANSSHIPMENT_COST_TRUCK, False)
        first_service_arrival = get_service_arrival(first_service_id)
        truck_travel_time = get_truck_travel_time(get_service_destination(first_service_id),get_service_origin(third_service_id))

        if first_service_arrival is not None and truck_travel_time is not None:
            truck_ID = get_truck_ID(get_service_destination(first_service_id),get_service_origin(third_service_id))
            new_row['service_ids'] = first_service_id + f', {truck_ID}, ' + third_service_id
            new_row['first_service_departure'] = row['first_service_departure']
            truck_departure = first_service_arrival + truck_deaprture_window  # Truck departure one hour after first service arrival
            truck_arrival = truck_departure + truck_travel_time
            third_service_departure = get_service_departure(third_service_id)  # Third service departure
            third_service_arrival = get_service_arrival(third_service_id)
            new_row['last_service_arrival'] = third_service_arrival
            new_row['storage_time'] = truck_deaprture_window-truck_loading_time + third_service_departure - truck_arrival-LOADING_TIME
            new_row['storage_cost'] = new_row['storage_time']*STORAGE_COST_PER_HOUR
            new_row['transshipment_cost'] = updated_transshipment_cost
            new_row['total_transport_cost'] = get_service_cost(first_service_id)+get_service_cost(third_service_id) + get_truck_travel_cost(get_service_destination(first_service_id),get_service_origin(third_service_id))
            new_row['total_cost'] = new_row['total_transport_cost']+new_row['storage_cost']+row['transshipment_cost']
            new_row['service_capacity'] = min(get_service_capacity(first_service_id),get_service_capacity(third_service_id))
            new_rows.append(new_row)
        
        # Replace the third service with "Truck"
        new_row = row.copy()
        third_mode = third_service_id[:-2]
        updated_transshipment_cost = correct_transshipment_costs(row['transshipment_cost'], third_mode, TRANSSHIPMENT_COST_BARGE, TRANSSHIPMENT_COST_TRAIN, TRANSSHIPMENT_COST_TRUCK, True)
        second_service_arrival = get_service_arrival(second_service_id)
        truck_travel_time = get_truck_travel_time(get_service_origin(third_service_id),get_service_destination(third_service_id))
        if second_service_arrival is not None and truck_travel_time is not None:
            truck_ID = get_truck_ID(get_service_origin(third_service_id),get_service_destination(third_service_id))
            new_row['service_ids'] = first_service_id + ', ' + second_service_id + f', {truck_ID}'
            new_row['first_service_departure'] = row['first_service_departure']
            truck_departure = second_service_arrival + truck_deaprture_window  
            truck_arrival = truck_departure + truck_travel_time
            new_row['last_service_arrival'] = truck_arrival
            new_row['storage_time'] = (get_service_departure(second_service_id) - get_service_arrival(first_service_id)-LOADING_TIME) + (truck_deaprture_window-truck_loading_time)
            new_row['storage_cost'] = new_row['storage_time']*STORAGE_COST_PER_HOUR
            new_row['transshipment_cost'] = updated_transshipment_cost
            new_row['total_transport_cost'] = get_service_cost(first_service_id)+get_service_cost(second_service_id) + get_truck_travel_cost(get_service_origin(third_service_id),get_service_destination(third_service_id))
            new_row['total_cost'] = new_row['total_transport_cost']+new_row['storage_cost']+row['transshipment_cost']
            new_row['service_capacity'] = min(get_service_capacity(first_service_id),get_service_capacity(second_service_id))
            new_row['last_service_mode'] = 'Truck'
            new_rows.append(new_row)

# Add the new rows to the dataframe
new_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
new_df['Transshipment Terminal(s)'] = new_df['path'].apply(extract_intermediary_points)
new_df['Transshipment Terminal(s)'] = new_df['Transshipment Terminal(s)'].apply(lambda s: '0' if s == '' else str(s))

# Add loading and unloading costs
add_loading_unloading_costs(new_df, TRANSSHIPMENT_COST_BARGE, TRANSSHIPMENT_COST_TRAIN, TRANSSHIPMENT_COST_TRUCK)

new_df.to_csv(f'{data_path}\\{possible_paths_fn}')

c:\Users\Satrya Dewantara\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
